### Set up the panel for data to be merged in

In [1]:
clear
cd
// There are 13213 signal intersections
global intersect = 13213+1
global periods_per_year = 12
global obs = $periods_per_year*7*($intersect)


display $obs
set obs $obs



D:\Box Sync\Policy Evaluation\dofiles




1109976

number of observations (_N) was 0, now 1,109,976


In [2]:
egen intersection_id = seq(), from(1) to($intersect)
sort intersection_id

egen month = seq(), from(1) to (12) by(intersection_id)
egen year = seq(), from(2012) to (2018) block($periods_per_year) 

gen monthly = ym(year, month)
format monthly %tm

//These are outside of the range of our data
drop if month < 7 & year == 2012
drop if month > 9 & year == 2018








(79,284 observations deleted)

(39,642 observations deleted)


### Merge in Intersection data from "Signal intersection.ipynb"


In [3]:
// Intersection level data
mmerge intersection_id using "..\working_data\signal_intersection.dta", ///
type(n:1) ///
unmatched(using) ///
umatch(intersection_id)



-------------------------------------------------------------------------------
merge specs          |
       matching type | n:1
  mv's on match vars | none
  unmatched obs from | using
---------------------+---------------------------------------------------------
  master        file | <data in memory not named>
                 obs | 991050
                vars |      4
          match vars | intersection_id  (not a key)
  -------------------+---------------------------------------------------------
  using         file | ..\working_data\signal_intersection.dta
                 obs |  13209
                vars |     35
          match vars | intersection_id  (key)
---------------------+---------------------------------------------------------
variable intersection_id does not uniquely identify observations in the master data
result          file | <data in memory not named>
                 obs | 990675
                vars |     40  (including _merge)
         ------------+----

In [4]:
describe


Contains data
  obs:       990,675                          
 vars:            39                          
 size:   429,952,950                          
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
              storage   display    value
variable name   type    format     label      variable label
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
intersection_id int     %8.0g                 
month           byte    %8.0g                 
year            int     %8.0g                 
monthly         int     %tm                   
index           int     %12.0g            

### Create borough dummies


In [5]:
// Create borough dummies
tab bname,m
gen bronx = (bname == "Bronx")
gen brooklyn = (bname == "Brooklyn")
gen manhattan = (bname == "Manhattan")
gen queens = (bname == "Queens")
gen statenisland = (bname == "Staten Island")



        bname |      Freq.     Percent        Cum.
--------------+-----------------------------------
        Bronx |    127,800       12.90       12.90
     Brooklyn |    352,200       35.55       48.45
    Manhattan |    215,325       21.74       70.19
       Queens |    248,400       25.07       95.26
Staten Island |     46,950        4.74      100.00
--------------+-----------------------------------
        Total |    990,675      100.00







### Lion Street data 

In [6]:
// Lion Street data 
// Variables: Number_Tra Number_Par Number_Tot TrafDir distance_to_Street nearest_Street
rename Number_Tra num_traveling_lanes
rename Number_Par num_parking_lanes
rename Number_Tot num_total_lanes
rename TrafDir trafdir

In [7]:
replace num_traveling_lanes = "" if num_traveling_lanes == "None"
replace num_parking_lanes = "" if num_parking_lanes == "None"
replace num_total_lanes = "" if num_total_lanes == "None"
destring num_traveling_lanes num_parking_lanes num_total_lanes, replace


(120,675 real changes made)

(238,500 real changes made)

(120,750 real changes made)

num_traveling_lanes: all characters numeric; replaced as byte
(120675 missing values generated)
num_parking_lanes: all characters numeric; replaced as byte
(238500 missing values generated)
num_total_lanes: all characters numeric; replaced as byte
(120750 missing values generated)


In [8]:
replace trafdir = "1" if trafdir == "W"
replace trafdir = "2" if trafdir == "A"
replace trafdir = "3" if trafdir == "T"
replace trafdir = "4" if trafdir == "P"
replace trafdir = "" if trafdir == "None"
destring trafdir, replace

label define trafdir1 1 "With: One-way street, traffic flows with the segment's directionality" ///
2 "Against: One-way street, traffic flows from against the segment's directionality" ///
3 "Two-Way: Traffic flows in both directions" ///
4 "Pedestrian path: Non-vehicular"
label values trafdir trafdir1


(208,350 real changes made)

(198,975 real changes made)

(515,250 real changes made)

(5,925 real changes made)

(62,175 real changes made)

trafdir: all characters numeric; replaced as byte
(62175 missing values generated)




In [9]:
// Problem 1: Missing intersection data
// Solution 1: Inpute it as mean
codebook num_traveling_lanes
gen mi_number_travel_lanes = mi(num_traveling_lanes), after(num_traveling_lanes)
sum num_traveling_lanes
replace num_traveling_lanes = r(mean) if mi_number_travel_lanes == 1

codebook num_parking_lanes
gen mi_number_park_lanes = mi(num_parking_lanes), after(num_parking_lanes)
sum num_parking_lanes
replace num_parking_lanes = 0 if mi_number_park_lanes == 1



---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
num_traveling_lanes                                                                                                                                                                                                                                 (unlabeled)
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

                  type:  numeric (byte)

                 range:  [1,8]                        units:  1
         unique values:  7                        missing .:  120,675/990,675

            tabulation:  Freq.  Value
       

### Nearest Bike Route data

In [10]:
// Nearest Bike Route data
// Variables nearest_bikeroute distance_to_bikeroute
destring nearest_bikeroute,replace
mmerge nearest_bikeroute using "..\working_data\bike_routes_2263.dta", ///
type(n:1) ///
unmatched(master) ///
umatch(objectid_1) ///
ukeep(date_instd tf_facilit)


nearest_bikeroute already numeric; no replace


-------------------------------------------------------------------------------
merge specs          |
       matching type | n:1
  mv's on match vars | none
  unmatched obs from | master
---------------------+---------------------------------------------------------
  master        file | <data in memory not named>
                 obs | 990675
                vars |     45
          match vars | nearest_bikeroute  (not a key)
  -------------------+---------------------------------------------------------
  using         file | ..\working_data\bike_routes_2263.dta
                 obs |  14980
                vars |      3  (selection via udrop/ukeep)
          match vars | objectid_1  (key)
---------------------+---------------------------------------------------------
variable nearest_bikeroute does not uniquely identify observations in the master data
result          file | <data in memory not named>
                 obs | 990675
   

In [11]:
gen bike_route_install_dt = date(date_instd, "YMD"), after(date_instd)
format bike_route_install_dt %td
gen bike_route_install_mt = mofd(bike_route_install_dt)
format bike_route_install_mt %tm
gen bike_route_install_qt = qofd(bike_route_install_dt)
format bike_route_install_qt %tq
drop date_instd


In [12]:
// Determine if collision is on bike route
sum distance_to_bikeroute, detail
sum distance_to_bikeroute if distance_to_bikeroute < r(p50) , detail

gen bike_route_ever = distance_to_bikeroute <= 3
tab bike_route_ever
label variable bike_route_ever "Indicates that bike route passes through this intersection (<3 ft)"


// Timevarient bike route indicator
gen bike_route_tv = 0 
replace bike_route_tv = 1 if monthly >= bike_route_install_mt & bike_route_ever == 1

label variable bike_route_tv "Indicates that bike route passes through this intersection timevarient(<3 ft)"



                    distance_to_bikeroute
-------------------------------------------------------------
      Percentiles      Smallest
 1%            0              0
 5%            0              0
10%            1              0       Obs             990,675
25%            3              0       Sum of Wgt.     990,675

50%          537                      Mean           1022.982
                        Largest       Std. Dev.      1366.029
75%         1415           8907
90%         2871           8907       Variance        1866035
95%         4039           8907       Skewness       2.002712
99%         6101           8907       Kurtosis       7.321187


                    distance_to_bikeroute
-------------------------------------------------------------
      Percentiles      Smallest
 1%            0              0
 5%            0              0
10%            0              0       Obs             495,000
25%            1              0       Sum of Wgt.     495,000

50% 

## Truck Route

In [13]:
// Truck route
// Determine if collision is on truck route
sum distance_to_truckroute, detail
sum distance_to_truckroute if distance_to_truckroute < r(p50) , detail
sum distance_to_truckroute if distance_to_truckroute < r(p90) , detail
gen truck_route = distance_to_truckroute <= 3
tab truck_route

label variable truck_route "Indicates that truck route passes through this intersection (<3 ft)"




                   distance_to_truckroute
-------------------------------------------------------------
      Percentiles      Smallest
 1%            2              0
 5%            2              0
10%            2              0       Obs             990,675
25%            3              0       Sum of Wgt.     990,675

50%          263                      Mean           551.0796
                        Largest       Std. Dev.      753.6798
75%          827           9622
90%         1570           9622       Variance       568033.3
95%         2118           9622       Skewness       2.078965
99%         3125           9622       Kurtosis       10.19282


                   distance_to_truckroute
-------------------------------------------------------------
      Percentiles      Smallest
 1%            1              0
 5%            2              0
10%            2              0       Obs             494,175
25%            2              0       Sum of Wgt.     494,175

50% 

### Left turn intervention

In [14]:
// Left turn calming interventions information
// Variables treatment_ completion nearest_LTC distance_to_LTC
rename treatment_ left_turn_treatment

gen left_turn_install_dt = date(completion, "YMD"), after(completion)
format left_turn_install_dt %td
gen int left_turn_install_mt = mofd(left_turn_install_dt), after(left_turn_install_dt)
format left_turn_install_mt %tm
gen int left_turn_install_qt = qofd(left_turn_install_dt), after(left_turn_install_dt)
format left_turn_install_qt %tq
drop completion

In [15]:
// Matching intersection to closest Left turn calming interventions
egen double left_turn_min = min(distance_to_LTC), by(nearest_LTC)
gen flag_left_turn_ever = .
replace flag_left_turn_ever = 1 if left_turn_min == distance_to_LTC
replace flag_left_turn_ever = 0 if mi(flag_left_turn_ever)

gen flag_left_turn = flag_left_turn_ever
replace flag_left_turn = 0 if flag_left_turn == 1 & monthly < left_turn_install_mt



(990,675 missing values generated)

(17,400 real changes made)

(973,275 real changes made)


(13,382 real changes made)


### Street improvement project

In [16]:
// Street Improv Project interventions information
// Variable DateComplt SIPProjTyp distance_to_StImpro nearest_StImpro
rename SIPProjTyp street_improv_treatment

gen street_improv_install_dt = date(DateComplt, "YMD"), after(DateComplt)
format street_improv_install_dt %td
gen int street_improv_install_mt = mofd(street_improv_install_dt), after(street_improv_install_dt)
format street_improv_install_mt %tm
gen int street_improv_install_qt = qofd(street_improv_install_dt), after(street_improv_install_dt)
format street_improv_install_qt %tq
drop DateComplt

In [17]:
// Matching intersection to closest street improvment interventions
egen double street_improv_min = min(distance_to_StImpro), by(nearest_StImpro)
gen flag_street_improv_ever = .
replace flag_street_improv_ever = 1 if street_improv_min == distance_to_StImpro
replace flag_street_improv_ever = 0 if mi(flag_street_improv_ever)

gen flag_street_improv = flag_street_improv_ever
replace flag_street_improv = 0 if flag_street_improv == 1 & monthly < street_improv_install_mt



(990,675 missing values generated)

(27,375 real changes made)

(963,300 real changes made)


(14,107 real changes made)


### Leading Pedestrian Interval Signal

In [18]:
// LPIS information
// Variable distance_to_LPIS nearest_LPIS date_insta
gen LPIS_install_dt = date(date_insta, "YMD")
format LPIS_install_dt %td
gen int LPIS_install_mt = mofd(LPIS_install_dt), after(LPIS_install_dt)
format LPIS_install_mt %tm
gen int LPIS_install_qt = qofd(LPIS_install_dt), after(LPIS_install_mt)
format LPIS_install_qt %tq 
gen LPIS_install_yr = year(LPIS_install_dt), after(LPIS_install_qt)
drop date_insta

In [19]:
// Matching intersection to closest LPIS intersection
egen double LPIS_min = min(distance_to_LPIS), by(nearest_LPIS)
gen flag_LPIS_ever = .
replace flag_LPIS_ever = 1 if LPIS_min == distance_to_LPIS
replace flag_LPIS_ever = 0 if mi(flag_LPIS_ever)

gen flag_LPIS = flag_LPIS_ever



(990,675 missing values generated)

(218,625 real changes made)

(772,050 real changes made)



In [20]:
// Problem2: Missing Install date on some LPIS IDs
sum LPIS_install_mt, detail format
codebook LPIS_install_mt 
tab nearest_LPIS if mi(LPIS_install_mt) & flag_LPIS == 1
tab intersection_id if mi(LPIS_install_mt) & flag_LPIS == 1
sum distance_to_LPIS if mi(LPIS_install_mt)

// Solution2: Drop these intersections
tab flag_LPIS if mi(LPIS_install_mt)
drop if mi(LPIS_install_mt) & flag_LPIS == 1
//0 observations deleted

// Replace with indicator with 0 if month was before LPIS installation
replace flag_LPIS = 0 if flag_LPIS == 1 & monthly < LPIS_install_mt
*145,199 real changes made



                       LPIS_install_mt
-------------------------------------------------------------
      Percentiles      Smallest
 1%    1899m11      1899m11
 5%     2004m3      1899m11
10%     2013m6      1899m11       Obs             990,675
25%    2015m11      1899m11       Sum of Wgt.     990,675

50%     2016m9                      Mean           2014m6
                        Largest       Std. Dev.      155.3959
75%     2017m9       2018m9
90%     2018m5       2018m9       Variance       24147.89
95%     2018m8       2018m9       Skewness      -7.819602
99%     2018m9       2018m9       Kurtosis       68.19454


---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
LPIS_install_mt                                                                                                 

In [21]:
// Problem3: Certain LPIS junctions have been installed 
// before the collisions datasets begin
// Removing junctions that cannot be analyzed because LPIS was done before 01jul2012
sum monthly, detail format
sum LPIS_install_mt, detail format

// Solution3: Drop these observations 
tab flag_LPIS if LPIS_install_mt < tm(2013m1)
tab intersection_id if LPIS_install_mt < tm(2013m1) & flag_LPIS_ever == 1
drop if LPIS_install_mt < tm(2013m1) & flag_LPIS_ever == 1
* 16,950 observations deleted



                           monthly
-------------------------------------------------------------
      Percentiles      Smallest
 1%     2012m7       2012m7
 5%    2012m10       2012m7
10%     2013m2       2012m7       Obs             990,675
25%     2014m1       2012m7       Sum of Wgt.     990,675

50%     2015m8                      Mean           2015m8
                        Largest       Std. Dev.      21.64872
75%     2017m3       2018m9
90%     2018m2       2018m9       Variance       468.6671
95%     2018m6       2018m9       Skewness              0
99%     2018m9       2018m9       Kurtosis       1.799573


                       LPIS_install_mt
-------------------------------------------------------------
      Percentiles      Smallest
 1%    1899m11      1899m11
 5%     2004m3      1899m11
10%     2013m6      1899m11       Obs             990,675
25%    2015m11      1899m11       Sum of Wgt.     990,675

50%     2016m9                      Mean           2014m6
        

       2825 |         75        0.44       59.73
       3005 |         75        0.44       60.18
       3140 |         75        0.44       60.62
       3455 |         75        0.44       61.06
       3561 |         75        0.44       61.50
       4090 |         75        0.44       61.95
       4340 |         75        0.44       62.39
       4701 |         75        0.44       62.83
       4735 |         75        0.44       63.27
       5042 |         75        0.44       63.72
       5054 |         75        0.44       64.16
       5276 |         75        0.44       64.60
       5278 |         75        0.44       65.04
       5285 |         75        0.44       65.49
       5314 |         75        0.44       65.93
       5345 |         75        0.44       66.37
       5346 |         75        0.44       66.81
       5348 |         75        0.44       67.26
       5363 |         75        0.44       67.70
       5385 |         75        0.44       68.14
       5398 |       

### Checking that panel is strongly balanced

In [22]:
duplicates tag intersection_id, gen(dup2)
tab dup2
drop dup2



Duplicates in terms of intersection_id


       dup2 |      Freq.     Percent        Cum.
------------+-----------------------------------
         74 |    973,725      100.00      100.00
------------+-----------------------------------
      Total |    973,725      100.00



In [23]:
label variable flag_LPIS "Indicates the intersection at the time they became an LPIS intervention"
label variable flag_LPIS_ever "Indicates the intersection if they ever received LPIS intervention"

### Determine if intersection is within 2000ft of LPIS intersection

In [24]:
mmerge intersection_id using "..\working_data\intersection_2000ft_LPIS_reshaped.dta", ///
type(n:1) ///
unmatched(master) ///
umatch(intersection_id)



-------------------------------------------------------------------------------
merge specs          |
       matching type | n:1
  mv's on match vars | none
  unmatched obs from | master
---------------------+---------------------------------------------------------
  master        file | <data in memory not named>
                 obs | 973725
                vars |     68
          match vars | intersection_id  (not a key)
  -------------------+---------------------------------------------------------
  using         file | ..\working_data\intersection_2000ft_LPIS_reshaped.dta
                 obs |  11762
                vars |     54
          match vars | intersection_id  (key)
---------------------+---------------------------------------------------------
variable intersection_id does not uniquely identify observations in the master data
result          file | <data in memory not named>
                 obs | 973725
                vars |    123  (including _merge)
         --

### Merge outcomes data

In [25]:
// Outcomes data
mmerge month year intersection_id using "..\working_data\collision_monthly.dta", ///
type(1:1) ///
unmatched(master)
// Reason that we did not match every obs from using
// is that we droped intersections in the 
// 1.Intersection level data merge process
// 2. Problem2 Certain LPIS junctions have been installed 
// before the collisions datasets begin


-------------------------------------------------------------------------------
merge specs          |
       matching type | 1:1
  mv's on match vars | none
  unmatched obs from | master
---------------------+---------------------------------------------------------
  master        file | <data in memory not named>
                 obs | 973725
                vars |    121
          match vars | month year intersection_id  (key)
  -------------------+---------------------------------------------------------
  using         file | ..\working_data\collision_monthly.dta
                 obs | 338263
                vars |     30
          match vars | month year intersection_id  (key)
---------------------+---------------------------------------------------------
result          file | <data in memory not named>
                 obs | 973725
                vars |    150  (including _merge)
         ------------+---------------------------------------------------------
              _m

### Fill in missing information

In [26]:
// Fill in '0' for intersection month year that did not have collisions
replace personsinjured = 0 if _merge == 1
replace personskilled = 0 if _merge == 1
replace pedestriansinjured = 0 if _merge == 1
replace pedestrianskilled = 0 if _merge == 1
replace cyclistinjured = 0 if _merge == 1
replace cyclistkilled = 0 if _merge == 1
replace motoristinjured = 0 if _merge == 1
replace motoristkilled = 0 if _merge == 1
replace collision_count = 0 if mi(collision_count)

replace latenight_personsinjured = 0 if _merge == 1
replace latenight_personskilled = 0 if _merge == 1
replace latenight_pedestriansinjured = 0 if _merge == 1
replace latenight_pedestrianskilled = 0 if _merge == 1
replace latenight_cyclistinjured = 0 if _merge == 1
replace latenight_cyclistkilled = 0 if _merge == 1
replace latenight_motoristinjured = 0 if _merge == 1
replace latenight_motoristkilled = 0 if _merge == 1
replace latenight_collision_count = 0 if mi(latenight_collision_count)

replace day_personsinjured = 0 if _merge == 1
replace day_personskilled = 0 if _merge == 1
replace day_pedestriansinjured = 0 if _merge == 1
replace day_pedestrianskilled = 0 if _merge == 1
replace day_cyclistinjured = 0 if _merge == 1
replace day_cyclistkilled = 0 if _merge == 1
replace day_motoristinjured = 0 if _merge == 1
replace day_motoristkilled = 0 if _merge == 1
replace day_collision_count = 0 if mi(day_collision_count)

gen flag_collision = collision_count > 0
gen latenight_flag_collision = latenight_collision_count > 0
gen day_flag_collision = day_collision_count > 0


label variable personsinjured "No. of persons injured during that month"
label variable personskilled "No. of persons killed during that month"
label variable pedestriansinjured "No. of pedestrians injured during that month"
label variable pedestrianskilled "No. of pedestrians killed during that month"
label variable cyclistinjured "No. of cyclist injured during that month"
label variable cyclistkilled "No. of cyclist killed during that month"
label variable motoristinjured "No. of motorist injured during that month"
label variable motoristkilled "No. of motorist killed during that month"
label variable collision_count "No. of collisions occured during that month"
label variable flag_collision "At least one collision occurred that month"

label variable latenight_personsinjured "No. of persons injured during that month between 11pm - 5am"
label variable latenight_personskilled "No. of persons killed during that month between 11pm - 5am"
label variable latenight_pedestriansinjured "No. of pedestrians injured during that month between 11pm - 5am"
label variable latenight_pedestrianskilled "No. of pedestrians killed during that month between 11pm - 5am"
label variable latenight_cyclistinjured "No. of cyclist injured during that month between 11pm - 5am"
label variable latenight_cyclistkilled "No. of cyclist killed during that month between 11pm - 5am"
label variable latenight_motoristinjured "No. of motorist injured during that month between 11pm - 5am"
label variable latenight_motoristkilled "No. of motorist killed during that month between 11pm - 5am"
label variable latenight_collision_count "No. of collisions occured during that month between 11pm - 5am"
label variable latenight_flag_collision " At least one collision occurred that month between 11pm - 5am"

label variable day_personsinjured "No. of persons injured during that month between 5am - 11pm"
label variable day_personskilled "No. of persons killed during that month between 5am - 11pm"
label variable day_pedestriansinjured "No. of pedestrians injured during that month between 5am - 11pm"
label variable day_pedestrianskilled "No. of pedestrians killed during that month between 5am - 11pm"
label variable day_cyclistinjured "No. of cyclist injured during that month between 5am - 11pm"
label variable day_cyclistkilled "No. of cyclist killed during that month between 5am - 11pm"
label variable day_motoristinjured "No. of motorist injured during that month between 5am - 11pm"
label variable day_motoristkilled "No. of motorist killed during that month between 5am - 11pm"
label variable day_collision_count "No. of collisions occured during that month between 5am - 11pm"
label variable day_flag_collision " At least one collision occurred that month between 5am - 11pm"

drop _merge


(646,549 real changes made)

(646,549 real changes made)

(646,549 real changes made)

(646,549 real changes made)

(646,549 real changes made)

(646,549 real changes made)

(646,549 real changes made)

(646,549 real changes made)

(646,549 real changes made)

(646,549 real changes made)

(646,549 real changes made)

(646,549 real changes made)

(646,549 real changes made)

(646,549 real changes made)

(646,549 real changes made)

(646,549 real changes made)

(646,549 real changes made)

(646,549 real changes made)

(646,549 real changes made)

(646,549 real changes made)

(646,549 real changes made)

(646,549 real changes made)

(646,549 real changes made)

(646,549 real changes made)

(646,549 real changes made)

(646,549 real changes made)

(646,549 real changes made)




































In [27]:
save "..\working_data\analytical_file_panel_mt.dta",replace

file ..\working_data\analytical_file_panel_mt.dta saved
